In [1]:
using DataFrames
using DataFramesMeta
using GLM
using CSV

In [2]:
# TODO
# figure out how to read in types
#data = vec(readdlm("coltypes.csv", ',', String))
#add type validations df::DataFrame
# add exceptions for unique ids, etc
# deal with missings
@doc """
make dataset
"""
function make_dataset(df, _id, label, case, control, X)
    df2 = @subset(df, $label  .== case .|| $label .== control)
    _ids = df2[:, _id]
    labels = df2[:, label]
    df2 = df2[:, vcat(label, X)]
    df2 = @transform!(df2, $label = ($label .== case))
    return df2, _ids, labels 
end

make_dataset

In [3]:
function fit_logit(df, label)
    X = sum(Term.(Symbol.(names(df[:, Not(label)]))))
    logit = glm(Term(label) ~ X, df, Binomial(), LogitLink())
    return logit
end

fit_logit (generic function with 1 method)

In [4]:
function propensity_scores(df, _id, labels, model, _id_col_name, label_col_name, ps_col_name)
    ps = predict(model)
    return DataFrame(_id_col_name => _id, label_col_name => labels, ps_col_name => ps) 
end

propensity_scores (generic function with 1 method)

In [5]:
function merge_propensity_scores(df, ps_df, on)
    return leftjoin(df, ps_df, on = on)
end

merge_propensity_scores (generic function with 1 method)

In [18]:
types=[Float64, String, String, String, String]
file = "example_data/titanic.txt"
df = CSV.read(file, DataFrame; delim='\t', header=true, types=types)
df2, _ids, labels = make_dataset(df, "id", "Survived","Yes","No",["Class","Sex","Age"])
logit = fit_logit(df2,:Survived)
ps = propensity_scores(df2, _ids, labels, logit, "id", "Survived", "propensityScore")
merge_propensity_scores(df, ps, ["id","Survived"])

,id,Class,Sex,Age,Survived,propensityScore
,Float64,String,String,String,String,Float64?
1,1548.0,1st,Male,Adult,Yes,0.407038
2,2037.0,2nd,Female,Adult,Yes,0.73609
3,95.0,1st,Male,Adult,No,0.407038
4,678.0,3rd,Male,Adult,No,0.103959
5,1632.0,3rd,Male,Adult,Yes,0.103959
6,492.0,3rd,Male,Adult,No,0.103959
7,773.0,Crew,Male,Adult,No,0.2255
8,475.0,3rd,Male,Adult,No,0.103959
9,1023.0,Crew,Male,Adult,No,0.2255


In [82]:
function get_matches(df, n, _id_col_name, label_col_name, ps_col_name, case, control, caliper)
    cases = @subset(df, $label_col_name  .== case)
    controls = @subset(df, $label_col_name  .== control)
    controls = controls[:, "propensityScore"]
    for (idx, row) in enumerate(eachrow(cases))
        ps = row[ps_col_name]
        diffs = (abs.(controls .- ps))
        diffs = sort!(diffs[diffs .<= caliper])
        if length(diffs) > 0
            match_idx = findall(x -> x < caliper, diffs)[1:n]
            deleteat!(controls, match_idx)
            print(diffs[match_idx])
        end
    end
end

get_matches (generic function with 1 method)

In [84]:
get_matches(ps, 1, "id", "Survived", "propensityScore", "Yes", "No", 0.02)

[0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0

[0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][

[0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.004337734805590188][0.0][0.0][0.0][0.0][0.010527251157794515][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0][0.0]

In [ ]:
KNN = @load KNNClassifier

In [ ]:
knn = KNN()

In [ ]:
evaluate(knn, ps[:,"propensityScore"])

In [ ]:
ps[:,"propensityScore"]

In [ ]:
info("KNNClassifier")

In [ ]:
X, y = @load_crabs

In [ ]:
using NearestNeighbors
data = rand(1, 5)
k = 1
points = rand(10)

kdtree = KDTree(data)
idxs, dists = knn(kdtree, points, k);

In [ ]:
# println(points)
println(idxs)

In [ ]:
poi